## Домашнее задание на тему: "Улучшение качества модели"

In [57]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import seaborn as sns
 
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [82]:
# загрузим данным
data = pd.read_csv(load_boston()['filename'], skiprows=1)

In [83]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [84]:
# отберем любые 7 регрессоров
y = data['MEDV']
X = data[['INDUS', 'DIS', 'PTRATIO', 'AGE', 'CRIM', 'RM', 'LSTAT']]

## Кросс-валидация

In [6]:
from sklearn.model_selection import cross_val_score

In [7]:
from sklearn.ensemble import RandomForestRegressor

In [8]:
from sklearn.linear_model import RidgeCV

In [9]:
from sklearn.linear_model import LinearRegression
max_depth = list(range(1,15)), min_samples_split = list(range(2,15))

In [93]:
# В качестве первого алгоритма рассмотрим RandomForestRegressor, кросс-валидация позволяет получить разные результаты качества модели,
# в зависимости от разбиения на обучающую и тестовые выборки
RndForestReg = RandomForestRegressor(max_depth = 10, min_samples_leaf = 4)
scores = cross_val_score(RndForestReg, X, y, cv = 10)

In [94]:
# средний score составил 48%
print(scores)
print(scores.mean())

[ 0.70736563  0.83348883  0.19449474  0.80362536  0.82882543  0.74447464
  0.51212124  0.32804988 -0.28006557  0.12668205]
0.4799062219392029


In [97]:
# вторым алгоритмом возьмем ридж-регрессию
Ridge = RidgeCV(normalize = True)
scores = cross_val_score(Ridge, X, y, cv = 10)

In [98]:
# средний score составил всего 25.5%
print(scores)
print(scores.mean())

[ 0.7585919   0.6495775  -0.53714239  0.55294125  0.64560166  0.69583477
  0.19598754  0.1052868  -0.99541077  0.48202866]
0.2553296916066966


In [99]:
# третьим алгоритмом возьмем обычную линейную регрессию
Lin = LinearRegression(normalize = True)
scores = cross_val_score(Lin, X, y, cv = 10)
print(scores)
print(scores.mean()) 
# score составил 20.1% В целом, мы видим большой разброс значений на разных этапах кросс-валидации: от нулевых значений до 74%.

[ 0.72913547  0.6050246  -0.84640355  0.58414635  0.63579191  0.73611571
  0.14773451  0.04383579 -1.06052643  0.43166592]
0.20065202857237002


## GridSearchCV

In [15]:
from sklearn.model_selection import GridSearchCV

In [100]:
# GridSearchCV позволяет перебирать параметры метода, для регрессии случайного леса будем перебирать глубину дерева и кол-во объектов в листах
param_grid = {'max_depth':list(range(1,10)),
              'min_samples_leaf':list(range(1,10))
}
RFR = RandomForestRegressor()
grid = GridSearchCV(RFR, param_grid, cv=10, n_jobs = -1)

grid.fit(X,y)

GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9]})

In [101]:
# score немного увеличился для модели
print(grid.best_score_)
print(grid.best_estimator_)

0.483822803495916
RandomForestRegressor(max_depth=6, min_samples_leaf=4)


In [113]:
# для ридж-регрессии будем подбирать коэффициент альфа и использовать нормализованные и ненормализованные данные
alphas = np.logspace(-10, 10, 100)
param_grid = {'normalize':[True, False]}         
Ridge = RidgeCV(alphas=alphas)
grid = GridSearchCV(Ridge, param_grid = param_grid, cv=10)
grid.fit(X,y)

GridSearchCV(cv=10,
             estimator=RidgeCV(alphas=array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.71858824e-07, 4.32876128e-07, 6.89261210e-07...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10])),
             param_grid={'normalize': [True, False]})

In [114]:
# Score немного снизился по сравнению с кросс-валидацией: видимо, границы для подбора альфа были подобраны неудачно
print(grid.best_score_)
print(grid.best_estimator_)

0.23048629721067143
RidgeCV(alphas=array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.71858824e-07, 4.32876128e-07, 6.89261210e-07,
       1.09749877e-06, 1.74752840e-0...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10]),
        normalize=True)


## RandomizedSearch

In [22]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import  Lasso

In [115]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

In [116]:
# RandomizedSearchCV - выбирает случайные значения из параметров и пробует запускать обучение с ними, что обеспечивает более
# быстрое обучение при высоком score
# возьмем три алгоритма - RandomForestRegressor, Лассо и Ридж-регрессию

models=[
      {'name':'RF',"model": RandomForestRegressor(), 'params':{'max_depth': list(range(1,15)), 'min_samples_leaf': list(range(2,10))}},  
      {'name':'Ls',"model" : Lasso() , 'params':{'alpha':list(2**np.linspace(-10,10,100)), 'normalize':[True, False]}},
      {'name': 'Rd', "model" : RidgeCV(alphas=alphas), 'params': {'normalize':[True, False]}}
]

res=[]
for v in  models:
    res.append((v['name'], RandomizedSearchCV(v['model'], v['params'], cv=10).fit(X_train, y_train)))

In [117]:
# результаты потрясающие, Score увеличился с неудовлетворительных значений (20-48%) до 86%
for r in res:
    print(r[0], r[1].best_score_, r[1].best_params_)

RF 0.8609984020857434 {'min_samples_leaf': 3, 'max_depth': 6}
Ls 0.6956296085049181 {'normalize': False, 'alpha': 0.021262343752724643}
Rd 0.69910743867449 {'normalize': True}


In [119]:
# Проверим качество моделей на отложенной выборке. По-прежнему регрессия случайного леса дает лучший результат.
for r in res:
    print(r[0], r[1].best_estimator_.score(X_valid, y_valid))

RF 0.6373779294095769
Ls 0.5563031798193988
Rd 0.5584632902321816
